# ニューラルネットーワークに基づく(実数)MIMO検出

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wadayama/MIKA2019/blob/master/MIMO.ipynb)

本ノートブックでは、ニューラルネットワークにより実数体上のMIMO検出問題を行う。概要は次のとおり。
* $H \in \mathbb{R}^{4 \times 4}$: 干渉行列 (各要素は平均0, 分散1のガウス分布に従う)
* $y = x H + w$: 受信ベクトル(行ベクトル)
* 注意: テキスト(デフォルトで列ベクトル)とは異なっており、行と列が入れ替わっている。本ノートの中ではベクトルはデフォルトで行ベクトル
* $x \in \{+1, -1 \}^4$
* $w \in \mathbb{R}^4$: 各要素が平均0、分散$\sigma^2$のガウス分布に従う乱数ベクトル
* 目標は、受信ベクトルである$y$から送信ベクトル$x$を可能な限り正しく推定すること

## 必要なパッケージのインポート

In [ ]:
import torch # テンソル計算など
import torch.nn as nn  # ネットワーク構築用
import torch.optim as optim  # 最適化関数

## グローバル定数の設定

In [ ]:
mbs = 50 # ミニバッチサイズ
noise_std = 0.5 # 通信路において重畳される加法的白色ガウス雑音の標準偏差 (sigma)
n = 4 # アンテナ数
h = 50 # 隠れ層のユニット数
H = torch.normal(mean=torch.zeros(n, n), std=1.0) # 干渉行列
adam_lr = 0.001 # Adamの学習率

## 干渉行列の確認

In [ ]:
print(H)

## ネットワークの定義

In [ ]:
class Net(nn.Module): # nn.Module を継承
    def __init__(self): # コンストラクタ
        super(Net, self).__init__()
        self.detector = nn.Sequential(
            nn.Linear(n, h),  # W_1, b_1,
            nn.ReLU(), # 活性化関数としてReLUを利用
            nn.Linear(h, h), # W_2, b_2
            nn.ReLU(),
            nn.Linear(h, n)  # W_3, b_3
        )
    def forward(self, x): # 推論計算をforwardに書く
        x = self.detector(x)
        x = torch.tanh(x) # x \in {+1,-1}^4 なので、最終層はtanhを利用
        return x

## ミニバッチ生成関数

In [ ]:
def gen_minibatch():
    x = 1.0 - 2.0 * torch.randint(0, 2, (mbs, n)) # 送信ベクトル x をランダムに生成
    x = x.float()
    w = torch.normal(mean=torch.zeros(mbs, n), std = noise_std) # 加法的白色ガウス雑音の生成
    y = x @ H.t() + w # @は行列ベクトルの積
    return x, y

## ミニバッチ生成関数の実行例

In [ ]:
x, y = gen_minibatch()
print('x = ', x)
print('y = ', y)

## 訓練ループ

In [ ]:
model = Net() # ネットワークインスタンス生成
loss_func = nn.MSELoss() # 損失関数の指定(二乗損失関数)
optimizer = optim.Adam(model.parameters(), lr=adam_lr) # オプティマイザの指定(Adamを利用)
for i in range(10000):
    x, y = gen_minibatch() # ミニバッチの生成
    optimizer.zero_grad()  # オプティマイザの勾配情報初期化
    estimate = model(y)  # 推論計算
    loss = loss_func(x, estimate)  # 損失値の計算
    loss.backward()  # 誤差逆伝播法(後ろ向き計算の実行)
    optimizer.step()  # 学習可能パラメータの更新
    if i % 1000 == 0:
        print('i =', i, 'loss =', loss.item())

## 学習結果の確認

In [ ]:
mbs = 1
x, y = gen_minibatch()
print('x = ', x)
print('y = ', y)
x_hat = model(y)
print('x_hat = ', x_hat)

## ゼロフォーシング等化を試す

In [ ]:
Hinv = torch.inverse(H.t()) # H の逆行列を生成
x_hat_zero = y @ Hinv # 受信ベクトルに逆行列を乗じる
print('x_hat_zero = ', x_hat_zero)

## シンボル誤り率を測定する (ニューラル検出器)

In [ ]:
total_syms = 0
error_syms = 0
num_loops  = 1000
mbs = 1
with torch.no_grad(): # 学習しない場合は計算グラフの構築をする必要がないので、torch.no_grad()で推論計算部分を包む
    for i in range(num_loops):
        x, y = gen_minibatch()
        x_hat = model(y)
        total_syms += n
        error_syms += (torch.sign(x_hat) != x).sum().item() # 硬判定して送信ベクトルと比較
print('total_syms = ', total_syms)
print('error_syms = ', error_syms)
print('error prob = ', error_syms/total_syms)

## シンボル誤り率を測定する(ZF検出器)

In [ ]:
total_syms = 0
error_syms = 0
num_loops  = 1000
mbs = 1
Hinv = torch.inverse(H.t())
for i in range(num_loops):
    x, y = gen_minibatch()
    x_hat = y @ Hinv
    total_syms += n
    error_syms += (torch.sign(x_hat) != x).sum().item()
print('total_syms = ', total_syms)
print('error_syms = ', error_syms)
print('error prob = ', error_syms/total_syms)